Der Floyd-Warshall löst das **All-Pairs-Shortest-Path-Problem** in gerichteten Graphen mit konservativen Kantengewichten (also keine Zyklen mit negativem Gewicht), d.h. für jedes geordnete Knotenpaar wird ein kürzester, gerichteter Pfad zur Verbindung der Knoten des Paares gefunden (und zwar für jede Richtung, also von x nach y und von y nach x).

Der Algo verwendet auf recht elegante Art die folgende Beobachtung: Wenn ein Knoten k auf dem kürzesten Pfad von x nach y liegt, dann sind die Pfade von x nach k und von k nach y auch kürzeste Wege! Man kann also "größere" optimale Pfade aus "kleineren" optimalen Pfaden zusammensetzen!

Der Algo arbeitet dazu mit zwei Matrizen: shortest und pred. Beide Matrizen benötigen im Grunde nur zwei Dimensionen (jeweils die Knoten), verändern aber ihren Inhalt in jeder Runde des Algos. Wir führen deshalb in der Einstiegsversion des Algos einen dritten Index mit, der den Zustand der Matrizen in jeder Runde leicht nachvollziehbar macht.

Achtung: der Algo arbeitet bei n Knoten intern mit den Knotennummern 0..n-1, bei der Ausgabe der Vorgänger-Matrix verwendet er aber die Nummern 1..n (so, wie in der Vorlesung und im Buch). Bedenken Sie das, wenn Sie sich den Code genauer anschauen, um ihn zu verstehen.

Die Matrizen sind hier in Python übrigens als Listen von Listen von Listen umgesetzt, das geht (mit Hilfe der Bibliothek numpy) auch anders, aber an sich kann man das ganz gut verstehen.

In [80]:
from math import inf

# Helper funtion to print out a matrix more easily
# Show matrix:
def matrix(m,start=4):
    prefix = " "*start
    out = ""
    for i in range(0,len(m)): # x-Dimension
        out += "\n" + prefix + str(m[i]) 
    return out

# Wir gehen im Algo davon aus, dass der Graph bereits als nxn-Matrix gegeben ist.
# Ignore the parameter special for now
def floyd_warshall(g,debug=False):
    shortest = []
    pred = []
    
    n = len(g)
    
    # Initialisierung of shortest
    shortest.append(g) # We have a list of a matrix (which is logically a 3-dimensional matrix)
    # print(shortest[0][0][0]) # see how we can access it!
    
    # Initialisierung of pred, we use inf instead of "-" to show that we don't have a path from u to v
    pred_init = []
    for u in range(0,n):
        pred_row = []
        for v in range(0,n):
            pred_row.append(u+1 if g[u][v] != inf else inf)
        pred_init.append(pred_row)
    pred.append(pred_init)
    # (This can be done somewhat nicer with numpy)
    
    if debug:
        print("shortest[u,v,0] = ",matrix(shortest[0]),"\n")
        print("pred[u,v,0] = ",matrix(pred[0]),"\n")
                        
    
    # Note, to keep this somewhat simple locigally, the third index in our Scan is the first index here!
    for x in range(0,n):
        shortest_new = []
        pred_new = []
        for u in range(0,n):
            shortest_row = []
            pred_row = []
            for v in range(0,n):
                if shortest[x-1][u][v] > shortest[x-1][u][x] + shortest[x-1][x][v]:
                    shortest_row.append(shortest[x-1][u][x] + shortest[x-1][x][v])
                    pred_row.append(pred[x-1][x][v])
                else:
                    shortest_row.append(shortest[x-1][u][v])
                    pred_row.append(pred[x-1][u][v])
            shortest_new.append(shortest_row)
            pred_new.append(pred_row)
        shortest.append(shortest_new)
        pred.append(pred_new)        
        if debug:
            print("shortest[u,v,"+ str(x+1) +"] = ",matrix(shortest_new),"\n")
            print("pred[u,v,"+str(x+1)+"] = ",matrix(pred_new),"\n")
                        
    return shortest[-1],pred[-1]

    
# Define the graph you want to try out. inf represents INFINITY, of course, meaning: there is no edge
# Avoid negative cycles or your results will not be correct    
graph = [
    [0,   3, 8, inf],
    [inf, 0, inf, 1],
    [inf, 4, 0, inf],
    [2, inf, -5, 0]
]

print("Graph: ", matrix(graph),"\n")

shortest,pred = floyd_warshall(graph,debug=True)

print("\nKosten der kürzesten Wege:\n",matrix(shortest))
print("\nVorgängerknoten auf den kürzesten Wegen:\n",matrix(pred))

Graph:  
    [0, 3, 8, inf]
    [inf, 0, inf, 1]
    [inf, 4, 0, inf]
    [2, inf, -5, 0] 

shortest[u,v,0] =  
    [0, 3, 8, inf]
    [inf, 0, inf, 1]
    [inf, 4, 0, inf]
    [2, inf, -5, 0] 

pred[u,v,0] =  
    [1, 1, 1, inf]
    [inf, 2, inf, 2]
    [inf, 3, 3, inf]
    [4, inf, 4, 4] 

shortest[u,v,1] =  
    [0, 3, 8, inf]
    [inf, 0, inf, 1]
    [inf, 4, 0, inf]
    [2, 5, -5, 0] 

pred[u,v,1] =  
    [1, 1, 1, inf]
    [inf, 2, inf, 2]
    [inf, 3, 3, inf]
    [4, 1, 4, 4] 

shortest[u,v,2] =  
    [0, 3, 8, 4]
    [inf, 0, inf, 1]
    [inf, 4, 0, 5]
    [2, inf, -5, 0] 

pred[u,v,2] =  
    [1, 1, 1, 2]
    [inf, 2, inf, 2]
    [inf, 3, 3, 2]
    [4, inf, 4, 4] 

shortest[u,v,3] =  
    [0, 3, 8, inf]
    [inf, 0, inf, 1]
    [inf, 4, 0, inf]
    [2, -1, -5, 0] 

pred[u,v,3] =  
    [1, 1, 1, inf]
    [inf, 2, inf, 2]
    [inf, 3, 3, inf]
    [4, 3, 4, 4] 

shortest[u,v,4] =  
    [0, 3, -1, 4]
    [3, 0, -4, 1]
    [7, 4, 0, 5]
    [2, inf, -5, 0] 

pred[u,v,4] =  
    [1, 

Bitte, beachten Sie, dass wir versucht haben, den Algo möglichst nah an der Darstellung in Cormens Buch "Algorithms unlocked" zu halten. Wir haben deshalb eine kleine Unschärfe geerbt, man kann dort - wenn man den Graph als Matrix übergibt - nicht ohne Weiteres unterscheiden zwischen Kanten, die ein Gewicht von 0 haben und Einträgen, die deshalb eine 0 aufweisen, weil man in dem Knoten, bereits ist (also die Einträge entlang der Diagonale).

Wir haben oben entschieden, eine 0 immer als Kante zu interpretieren, deshalb hat jeder Knoten sich selbst als Vorgänger. Das ermöglicht eine einheitliche Behandlung, wenn man Loops (Kante von Knoten k zu sich selbst) mit negativem Gewicht (führt zu Problemen...) und Kanten ohne Gewicht modellieren möchte.

Die Konsequenz ist, dass wir entlang der Diagonale in den pred-Matrizen die jeweilige Knotennummer von Spalten bzw. Zeileneintrag haben (die sind dort ja gleich) und nicht, wie bei Cormen, "NULL" bzw. "-" (in unserer Vorlesung). Sie können das in der Klausur halten, wie sie wollen. Am besten verwenden Sie allerdings die Notation aus der Vorlesung, also ein "-".

Wir können übrigens den Index mit den Runden einfach weglassen. Das führt dann allerdings "zwischendrin" nicht immer zu den gleichen Matrizen, wie oben, am Ende aber schon.

Zeigen wir diesen Algo einmal (NICHT für das Üben und die Klausur verwenden!):

In [75]:
# Wir gehen im Algo davon aus, dass der Graph bereits als nxn-Matrix gegeben ist.
# Ignore the parameter special for now
def floyd_warshall_short(g,debug=False):
    shortest = []
    pred = []
    
    n = len(g)
        
    # Initialisierung of shortest
    shortest = g
    
    # Initialisierung of pred, we use inf instead of "-" to show that we don't have a path from u to v
    for u in range(0,n):
        pred_row = []
        for v in range(0,n):
            pred_row.append(u+1 if g[u][v] != inf else inf)
        pred.append(pred_row)
    
    
    if debug:
        print("Initialisierung: \n")
        print("shortest[u,v] = ",matrix(shortest),"\n")
        print("pred[u,v] = ",matrix(pred),"\n")
                        
    
    # Note, to keep this somewhat simple locigally, the third index in our Scan is the first index here!
    for x in range(0,n):
        for u in range(0,n):
            for v in range(0,n):
                if shortest[u][v] > shortest[u][x] + shortest[x][v]:
                    shortest[u][v] = shortest[u][x] + shortest[x][v]
                    pred[u][v] = pred[x][v]
        if debug:
            print("Round ",x+1,"\n")
            print("shortest[u,v] = ",matrix(shortest),"\n")
            print("pred[u,v] = ",matrix(pred),"\n")
                        
    return shortest,pred


In [76]:
# Probieren wir ihn aus mit dem Graph von oben:
graph1 = [
    [0,   3, 8, inf],
    [inf, 0, inf, 1],
    [inf, 4, 0, inf],
    [2, inf, -5, 0]
]

print("Graph: ",matrix(graph1),"\n")

shortest,pred = floyd_warshall_short(graph1, debug=True)

print("\nKosten der kürzesten Wege:\n",matrix(shortest))
print("\nVorgängerknoten auf den kürzesten Wegen:\n",matrix(pred))


Graph:  
    [0, 3, 8, inf]
    [inf, 0, inf, 1]
    [inf, 4, 0, inf]
    [2, inf, -5, 0] 

Initialisierung: 

shortest[u,v] =  
    [0, 3, 8, inf]
    [inf, 0, inf, 1]
    [inf, 4, 0, inf]
    [2, inf, -5, 0] 

pred[u,v] =  
    [1, 1, 1, inf]
    [inf, 2, inf, 2]
    [inf, 3, 3, inf]
    [4, inf, 4, 4] 

Round  1 

shortest[u,v] =  
    [0, 3, 8, inf]
    [inf, 0, inf, 1]
    [inf, 4, 0, inf]
    [2, 5, -5, 0] 

pred[u,v] =  
    [1, 1, 1, inf]
    [inf, 2, inf, 2]
    [inf, 3, 3, inf]
    [4, 1, 4, 4] 

Round  2 

shortest[u,v] =  
    [0, 3, 8, 4]
    [inf, 0, inf, 1]
    [inf, 4, 0, 5]
    [2, 5, -5, 0] 

pred[u,v] =  
    [1, 1, 1, 2]
    [inf, 2, inf, 2]
    [inf, 3, 3, 2]
    [4, 1, 4, 4] 

Round  3 

shortest[u,v] =  
    [0, 3, 8, 4]
    [inf, 0, inf, 1]
    [inf, 4, 0, 5]
    [2, -1, -5, 0] 

pred[u,v] =  
    [1, 1, 1, 2]
    [inf, 2, inf, 2]
    [inf, 3, 3, 2]
    [4, 3, 4, 4] 

Round  4 

shortest[u,v] =  
    [0, 3, -1, 4]
    [3, 0, -4, 1]
    [7, 4, 0, 5]
    [2, -1, -

In [87]:

# Probieren wir nun aus, ob der zweite Lauf des Algos für den folgenden
# Graphen zu einer Veränderung führt
graph2 = [
    [0,   3, 8, inf],
    [inf, 0, inf, 1],
    [inf, 4, 0, inf],
    [2, inf, -7, 0]
]

print("Graph: ",matrix(graph2),"\n")

shortest,pred = floyd_warshall_short(graph2,True)

print("\nKosten der kürzesten Wege:\n",matrix(shortest))
print("\nVorgängerknoten auf den kürzesten Wegen:\n",matrix(pred))

# We start now on the already modified graph
# (note that the edge information will become incorrect)

print("Graph: ",matrix(graph2),"\n")

shortest,pred = floyd_warshall_short(graph2,True)

print("\nKosten der kürzesten Wege:\n",matrix(shortest))
print("\nVorgängerknoten auf den kürzesten Wegen:\n",matrix(pred))




Graph:  
    [0, 3, 8, inf]
    [inf, 0, inf, 1]
    [inf, 4, 0, inf]
    [2, inf, -7, 0] 

Initialisierung: 

shortest[u,v] =  
    [0, 3, 8, inf]
    [inf, 0, inf, 1]
    [inf, 4, 0, inf]
    [2, inf, -7, 0] 

pred[u,v] =  
    [1, 1, 1, inf]
    [inf, 2, inf, 2]
    [inf, 3, 3, inf]
    [4, inf, 4, 4] 

Round  1 

shortest[u,v] =  
    [0, 3, 8, inf]
    [inf, 0, inf, 1]
    [inf, 4, 0, inf]
    [2, 5, -7, 0] 

pred[u,v] =  
    [1, 1, 1, inf]
    [inf, 2, inf, 2]
    [inf, 3, 3, inf]
    [4, 1, 4, 4] 

Round  2 

shortest[u,v] =  
    [0, 3, 8, 4]
    [inf, 0, inf, 1]
    [inf, 4, 0, 5]
    [2, 5, -7, 0] 

pred[u,v] =  
    [1, 1, 1, 2]
    [inf, 2, inf, 2]
    [inf, 3, 3, 2]
    [4, 1, 4, 4] 

Round  3 

shortest[u,v] =  
    [0, 3, 8, 4]
    [inf, 0, inf, 1]
    [inf, 4, 0, 5]
    [2, -3, -7, -2] 

pred[u,v] =  
    [1, 1, 1, 2]
    [inf, 2, inf, 2]
    [inf, 3, 3, 2]
    [4, 3, 4, 2] 

Round  4 

shortest[u,v] =  
    [0, 1, -3, 2]
    [3, -2, -6, -1]
    [7, 2, -2, 3]
    [0, -